## Question 1
Install Pipenv \
What's the version of pipenv you installed? \
Use `--version` to find out

In [33]:
!pip install pipenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pipenv --version

pipenv, version 2022.9.24


## Question 2
Use Pipenv to install Scikit-Learn version 1.0.2 \
What's the first hash for scikit-learn you get in Pipfile.lock?

In [35]:
!pipenv install scikit-learn==1.0.2

Installing scikit-learn==1.0.2...
Adding scikit-learn to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (97dec5)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [36]:
!pipenv shell

Launching subshell in virtual environment...
/content#  . /root/.local/share/virtualenvs/content-cQIIIOO2/bin/activate
(content) /content# 
Aborted!
^C


## Models

In [37]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import wget

In [ ]:
PREFIX = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework"
wget.download(PREFIX+"/model1.bin")
wget.download(PREFIX+"/dv.bin")

## Question 3
Let's use these models!

Write a script for loading these models with pickle. Score this client:

```
{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
```
What's the probability that this client will get a credit card?

In [40]:
import pickle

In [41]:
model = pickle.load(open('model1.bin', 'rb'))
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [42]:
dv = pickle.load(open('dv.bin', 'rb'))

In [43]:
X = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
X = dv.transform([X])
model.predict_proba(X)[0, 1]

0.16213414434326598

In [44]:
model.predict(X)[0]

0

In [45]:
model.predict_log_proba(X)[0, 1]

-1.8193312348936168

## Question 4
Now let's serve this model as a web service

Install Flask and gunicorn (or waitress, if you're on Windows) \
Write Flask code for serving the model
Now score this client using requests:

In [ ]:
from flask import Flask
from flask import request
from flask import jsonify


app = Flask('churn')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()

    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0, 1]
    churn = y_pred >= 0.5

    result = {
        'churn_probability': float(y_pred),
        'churn': bool(churn)
    }

    return jsonify(result)


if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

In [47]:
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]
churn = y_pred >= 0.5

result = {'churn_probability': float(y_pred),
          'churn': bool(churn)}

result

{'churn_probability': 0.9282218018527452, 'churn': True}

## Docker:

In [ ]:
# console:
# docker run python:3.9.12-slim
# docker pull svizor/zoomcamp-model:3.9.12-slim
# within docker:
# run: svizor/zoomcamp-model:3.9.12-slim
# open shell
# ls -> list of files: [model2.bin, dv.bin]
# pip install pipenv
# pipenv install sklearn==1.0.2
# pipenv shell
# run: python
# import pickle
# 


## Question 6 \
Let's run your docker container! \
After running it, score this client once again:

```
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
```



In [ ]:
model2 = pickle.load(open('model2.bin', 'rb'))
dv = pickle.load(open('dv.bin', 'rb'))

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
X = dv.transform([client])
model.predict_proba(X)[0, 1]